In [1]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    txt = f.read()
print("Total number of character: ", len(txt))
print(txt[:99])

Total number of character:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
import re
result = re.split(r'([,.?!:;_"()\']|--|\s)', txt);
print(result[:60])
print("\n")

result = [item for item in result if item.strip()]
print("number of token: ", len(result))
print(result[:60])

['I', ' ', 'HAD', ' ', 'always', ' ', 'thought', ' ', 'Jack', ' ', 'Gisburn', ' ', 'rather', ' ', 'a', ' ', 'cheap', ' ', 'genius', '--', 'though', ' ', 'a', ' ', 'good', ' ', 'fellow', ' ', 'enough', '--', 'so', ' ', 'it', ' ', 'was', ' ', 'no', ' ', 'great', ' ', 'surprise', ' ', 'to', ' ', 'me', ' ', 'to', ' ', 'hear', ' ', 'that', ',', '', ' ', 'in', ' ', 'the', ' ', 'height', ' ']


number of token:  4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I']


In [3]:
all_words = sorted(set(result))
## 加入特殊词源
all_words.extend(["<|endoftext|>", "<|unk|>"])
print("words number: ", len(all_words))
print(all_words[:60])

words number:  1132
['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft', 'Destroyed', 'Devonshire', 'Don', 'Dubarry', 'Emperors', 'Florence', 'For', 'Gallery', 'Gideon', 'Gisburn', 'Gisburns', 'Grafton', 'Greek', 'Grindle', 'Grindles', 'HAD', 'Had', 'Hang', 'Has', 'He', 'Her', 'Hermia', 'His', 'How', 'I', 'If', 'In', 'It', 'Jack', 'Jove', 'Just']


In [4]:
vocab = {token:integer for integer, token in enumerate(all_words)}
print("vocab length: ", len(vocab))

vocab length:  1132


In [5]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        result = re.split(r'([,.?!:;_"()\']|--|\s)', text)
        result = [item.strip() for item in result if item.strip()]
        idx = [self.str_to_int[i] for i in result]
        return idx

    def decode(self, idx):
        text = " ".join([self.int_to_str[i] for i in idx])
        text = re.sub(r'\s+([,.?!:;_"()\'])', r'\1', text)
        return text

In [6]:
simpletokenizer = SimpleTokenizerV1(vocab)
idx = simpletokenizer.encode(txt)
re_text = simpletokenizer.decode(idx)
print(idx[:100])
print("\n")
print(re_text[:100])

[53, 44, 149, 1003, 57, 38, 818, 115, 256, 486, 6, 1002, 115, 500, 435, 392, 6, 908, 585, 1077, 709, 508, 961, 1016, 663, 1016, 535, 987, 5, 568, 988, 538, 722, 549, 496, 5, 533, 514, 370, 549, 748, 5, 661, 115, 841, 1102, 5, 157, 397, 547, 568, 115, 1066, 727, 988, 84, 7, 3, 99, 53, 818, 1003, 585, 1120, 530, 208, 85, 734, 34, 7, 4, 1, 93, 538, 722, 549, 496, 1, 6, 987, 1077, 1089, 988, 1112, 242, 585, 7, 53, 244, 535, 67, 7, 37, 100, 6, 549, 602, 25, 897, 6]


I HAD always thought Jack Gisburn rather a cheap genius -- though a good fellow enough -- so it was 


In [7]:
## "Hello" don't exist in vocab, there is a error
simpletokenizer = SimpleTokenizerV1(vocab)
idx = simpletokenizer.encode("Hello World!")
re_text = simpletokenizer.decode(idx)
print(idx[:100])
print("\n")
print(re_text[:100])

KeyError: 'Hello'

In [8]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [9]:
## Class that can precess special token
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        result = re.split(r'([,.?!:;_"()\']|--|\s)', text)
        result = [item for item in result if item.strip()]
        result = [item if item in self.str_to_int else "<|unk|>" for item in result]
        idx = [self.str_to_int[i] for i in result]
        return idx

    def decode(self, idx):
        text = " ".join([self.int_to_str[i] for i in idx])
        text = re.sub(r'\s+([,.?!:;_"()\'])', r'\1', text)
        return text

In [11]:
test_text1 = "Hello, do you like hjsdhjs tea?"
test_text2 = "We will succeed in this test"
test_text = test_text1 + " <|endoftext|> " + test_text2
print(test_text)
simpletokenizer2 = SimpleTokenizerV2(vocab)
idx = simpletokenizer2.encode(test_text)
re_test_text = simpletokenizer2.decode(idx)
print(idx[:100])
print(re_test_text)

Hello, do you like hjsdhjs tea? <|endoftext|> We will succeed in this test
[1131, 5, 355, 1126, 628, 1131, 975, 10, 1130, 107, 1131, 1131, 568, 999, 1131]
<|unk|>, do you like <|unk|> tea? <|endoftext|> We <|unk|> <|unk|> in this <|unk|>


In [12]:
#### -----BPE----- ###
from importlib.metadata import version 
import tiktoken 
print("tiktoken version:", version("tiktoken"))


tiktoken version: 0.12.0


In [13]:
tokenizer = tiktoken.get_encoding("gpt2") 
test_text1 = "Hello, do you like hjsdhjs tea?"
test_text2 = "We will succeed in this test"
test_text = test_text1 + " <|endoftext|> " + test_text2
print(test_text)
integers = tokenizer.encode(test_text, allowed_special={"<|endoftext|>"}) 
print(integers)

Hello, do you like hjsdhjs tea? <|endoftext|> We will succeed in this test
[15496, 11, 466, 345, 588, 289, 8457, 34985, 8457, 8887, 30, 220, 50256, 775, 481, 6758, 287, 428, 1332]
